In [23]:
using LinearAlgebra, Printf

"""
Represents a quantum state in the computational basis
"""
struct QuantumState
    amplitudes::Vector{ComplexF64}
    n_qubits::Int

    function QuantumState(amplitudes::Vector{ComplexF64})
        n = length(amplitudes)
        # Check if the length is a power of 2
        ispow2(n) ||throw(ArgumentError("Number of amplitudes must be a power of 2"))
        
        # Normalize the state vector
        norm_factor = norm(amplitudes)
        !(norm_factor ≈ 0) || throw(ArgumentError("State vector cannot be zero"))
        new(amplitudes / norm_factor, Int(log2(n)))
    end
end

"""
Creates a computational basis state |i⟩ for given integer i
"""
function basis_state(i::Integer, n_qubits::Integer)
    !(i >= 2^n_qubits) || throw(ArgumentError("Index must be less than 2^n_qubits"))
    amplitudes = zeros(ComplexF64, 2^n_qubits)
    amplitudes[i + 1] = 1.0
    return QuantumState(amplitudes)
end

"""
Creates an equal superposition state
"""
function equal_superposition(n_qubits::Integer)
    dim = 2^n_qubits
    amplitudes = ones(ComplexF64, dim) / sqrt(dim)
    return QuantumState(amplitudes)
end

"""
Pretty print stuff
"""
function Base.show(io::IO, state::QuantumState)
    print(io, "Quantum state with $(state.n_qubits) qubits:\n")
    for (i, amplitude) in enumerate(state.amplitudes)
        if abs(amplitude) > 1e-10  # Only print non-zero amplitudes
            basis = string(i-1, base=2, pad=state.n_qubits)
            print(io, "  ($(round(amplitude, digits=3)))|$basis⟩\n")
        end
    end
end

"""
Applies a NOT gate to the specified qubit
"""
function apply_not!(state::QuantumState, target_qubit::Integer)
     !(target_qubit >= state.n_qubits) || !(target_qubit < 0) || throw(ArgumentError("Invalid target qubit"))
    
    n = state.n_qubits
    mask = 1 << target_qubit  # Bit mask for target qubit
    
    # Swap amplitudes where target qubit differs
    for i in 0:(2^n-1)
        # Check if we've already swapped this pair
        if (i & mask) == 0
            # j is i with the target qubit flipped
            j = i ⊻ mask
            # Swap amplitudes
            state.amplitudes[i+1], state.amplitudes[j+1] = 
                state.amplitudes[j+1], state.amplitudes[i+1]
        end
    end
    
    return state
end

"""
Applies a Hadamard gate to the specified qubit 
"""
function apply_hadamard!(state::QuantumState, target_qubit::Integer)
    !(target_qubit >= state.n_qubits) || !(target_qubit < 0) || throw(ArgumentError("Invalid target qubit"))
    
    n = state.n_qubits
    mask = 1 << target_qubit  # Bit mask for target qubit
    factor = 1/sqrt(2)
    
    # Create temporary array to store new amplitudes
    new_amplitudes = similar(state.amplitudes)
    
    # Process pairs of amplitudes that differ only in target qubit
    for i in 0:(2^n-1)
        if (i & mask) == 0  # Process each pair once
            j = i ⊻ mask    # j is i with the target qubit flipped
            
            # Apply Hadamard transformation to the pair
            new_amplitudes[i+1] = factor * (state.amplitudes[i+1] + state.amplitudes[j+1])
            new_amplitudes[j+1] = factor * (state.amplitudes[i+1] - state.amplitudes[j+1])
        end
    end
    
    # Update state with new amplitudes
    state.amplitudes .= new_amplitudes
    return state
end
function apply_cnot!(state::QuantumState, control_qubit::Integer, target_qubit::Integer)
    if control_qubit >= state.n_qubits || control_qubit < 0 ||
       target_qubit >= state.n_qubits || target_qubit < 0 ||
       control_qubit == target_qubit
        throw(ArgumentError("Invalid control or target qubit"))
    end
    
    n = state.n_qubits
    control_mask = 1 << control_qubit
    target_mask = 1 << target_qubit
    
    # Create temporary array to store new amplitudes
    new_amplitudes = copy(state.amplitudes)
    
    # Process all basis states
    for i in 0:(2^n-1)
        # Only apply operation if control qubit is 1
        if (i & control_mask) != 0
            # Flip the target qubit
            j = i ⊻ target_mask
            # Swap amplitudes
            new_amplitudes[i+1] = state.amplitudes[j+1]
        end
    end
    
    state.amplitudes .= new_amplitudes
    return state
end

apply_cnot! (generic function with 1 method)

In [4]:
state1 = QuantumState(collect(ComplexF64,1:8))
println(state1)
apply_not!(state1, 1)
println(state1)

Quantum state with 3 qubits:
  (0.07 + 0.0im)|000⟩
  (0.14 + 0.0im)|001⟩
  (0.21 + 0.0im)|010⟩
  (0.28 + 0.0im)|011⟩
  (0.35 + 0.0im)|100⟩
  (0.42 + 0.0im)|101⟩
  (0.49 + 0.0im)|110⟩
  (0.56 + 0.0im)|111⟩

Quantum state with 3 qubits:
  (0.21 + 0.0im)|000⟩
  (0.28 + 0.0im)|001⟩
  (0.07 + 0.0im)|010⟩
  (0.14 + 0.0im)|011⟩
  (0.49 + 0.0im)|100⟩
  (0.56 + 0.0im)|101⟩
  (0.35 + 0.0im)|110⟩
  (0.42 + 0.0im)|111⟩



true

In [3]:
teset_state=basis_state(0,4)
apply_hadamard!(teset_state,3)

Quantum state with 4 qubits:
  (0.707 + 0.0im)|0000⟩
  (0.707 + 0.0im)|1000⟩


In [24]:
state2 = QuantumState(ComplexF64[0.11,0.12,0.5,1,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,.21,.22,.23,.24])

print(state2)
state3=apply_cnot!(state2, 1,2)
print(state2)


Quantum state with 4 qubits:
  (0.084 + 0.0im)|0000⟩
  (0.092 + 0.0im)|0001⟩
  (0.383 + 0.0im)|0010⟩
  (0.767 + 0.0im)|0011⟩
  (0.1 + 0.0im)|0100⟩
  (0.107 + 0.0im)|0101⟩
  (0.115 + 0.0im)|0110⟩
  (0.123 + 0.0im)|0111⟩
  (0.13 + 0.0im)|1000⟩
  (0.138 + 0.0im)|1001⟩
  (0.146 + 0.0im)|1010⟩
  (0.153 + 0.0im)|1011⟩
  (0.161 + 0.0im)|1100⟩
  (0.169 + 0.0im)|1101⟩
  (0.176 + 0.0im)|1110⟩
  (0.184 + 0.0im)|1111⟩
Quantum state with 4 qubits:
  (0.084 + 0.0im)|0000⟩
  (0.092 + 0.0im)|0001⟩
  (0.115 + 0.0im)|0010⟩
  (0.123 + 0.0im)|0011⟩
  (0.1 + 0.0im)|0100⟩
  (0.107 + 0.0im)|0101⟩
  (0.383 + 0.0im)|0110⟩
  (0.767 + 0.0im)|0111⟩
  (0.13 + 0.0im)|1000⟩
  (0.138 + 0.0im)|1001⟩
  (0.176 + 0.0im)|1010⟩
  (0.184 + 0.0im)|1011⟩
  (0.161 + 0.0im)|1100⟩
  (0.169 + 0.0im)|1101⟩
  (0.146 + 0.0im)|1110⟩
  (0.153 + 0.0im)|1111⟩


In [27]:
function get_operation_matrix(gates::Vector{Tuple{Symbol, Vector{Int}}}, n_qubits::Int)
    dim = 2^n_qubits
    matrix = zeros(ComplexF64, dim, dim)
    
    # For each basis state
    for i in 0:(dim-1)
        # Create the basis state |i⟩
        state = basis_state(i, n_qubits)
        
        # Apply the sequence of gates
        for (gate_type, qubits) in gates
            if gate_type == :H
                apply_hadamard!(state, qubits[1])
            elseif gate_type == :CNOT
                apply_cnot!(state, qubits[1], qubits[2])
            else
                throw(ArgumentError("Unsupported gate type: $gate_type"))
            end
        end
        
        # The resulting state vector becomes the i-th column of the matrix
        matrix[:, i+1] = state.amplitudes
    end
    
    return matrix
end

"""
Prints a matrix with proper formatting for quantum operations
"""
function print_quantum_matrix(matrix::Matrix{ComplexF64})
    n = size(matrix, 1)
    println("Matrix representation ($(Int(log2(n))) qubits):")
    
    # Find the maximum width needed for pretty printing
    max_width = maximum(length(@sprintf("%.3f%+.3fi", real(x), imag(x))) for x in matrix)
    
    for i in 1:n
        print("  ")
        for j in 1:n
            val = matrix[i, j]
            if abs(val) < 1e-10
                print(lpad("0", max_width))
            else
                str = @sprintf("%.3f%+.3fi", real(val), imag(val))
                print(lpad(str, max_width))
            end
            print("  ")
        end
        println()
    end
end

# Example demonstrating CNOT₁₂H₁|00⟩ matrix construction
function demonstrate_matrix_construction()
    println("Demonstrating matrix construction for CNOT₁₂H₁:")
    
    # Define the sequence of gates: H₁ followed by CNOT₁₂
    # Note: Using 0-based indexing for consistency with previous implementation
    gates = [
       # (:H, [1]),           # H₁: Hadamard on qubit 1
        (:CNOT, [1, 2])      # CNOT₁₂: Control on qubit 1, target on qubit 0
    ]
    
    # Get the matrix representation
    matrix = get_operation_matrix(gates, 4)
    
    # Print the matrix
    print_quantum_matrix(matrix)
    
    # Verify that the matrix is unitary
    println("\nVerifying unitarity:")
    deviation = norm(matrix * matrix' - I)
    println("‖MM† - I‖ = $deviation")
    
    # Demonstrate the matrix action on |00⟩
    println("\nAction on |00⟩:")
    initial_state = [1.0 + 0.0im, 0, 0, 0]  # |00⟩ state
    final_state = matrix * initial_state
    
    println("Initial state: |00⟩")
    println("Final state:")
    for (i, amp) in enumerate(final_state)
        if abs(amp) > 1e-10
            basis = string(i-1, base=2, pad=2)
            println("  ($(round(amp, digits=3)))|$basis⟩")
        end
    end
    
    return matrix
end

# Run the demonstration
demonstrate_matrix_construction()

Demonstrating matrix construction for CNOT₁₂H₁:
Matrix representation (4 qubits):
  1.000+0.000i             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0  
             0  1.000+0.000i             0             0             0             0             0             0             0             0             0             0             0             0             0             0  
             0             0             0             0             0             0  1.000+0.000i             0             0             0             0             0             0             0             0             0  
             0             0             0             0             0             0             0  1.000+0.000i             0             0             0             0             0             0             0             0  
          

DimensionMismatch: DimensionMismatch: second dimension of A, 16, does not match length of x, 4

In [26]:
ComplexF64[0.11,0.12,0.5,1,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,.21,.22,.23,.24]/norm(ComplexF64[0.11,0.12,0.5,1,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,.21,.22,.23,.24])

16-element Vector{ComplexF64}:
 0.08432895302648113 + 0.0im
 0.09199522148343396 + 0.0im
  0.3833134228476415 + 0.0im
   0.766626845695283 + 0.0im
  0.0996614899403868 + 0.0im
 0.10732775839733964 + 0.0im
 0.11499402685429246 + 0.0im
  0.1226602953112453 + 0.0im
 0.13032656376819812 + 0.0im
 0.13799283222515094 + 0.0im
 0.14565910068210378 + 0.0im
 0.15332536913905662 + 0.0im
 0.16099163759600943 + 0.0im
 0.16865790605296227 + 0.0im
  0.1763241745099151 + 0.0im
 0.18399044296686792 + 0.0im